In [3]:
from cryptography import x509
from cryptography.hazmat.backends import default_backend
import os

# Função para carregar e exibir informações do certificado
def exibir_informacoes_certificado(caminho_certificado):
    try:
        with open(caminho_certificado, 'rb') as arquivo_certificado:
            conteudo_certificado = arquivo_certificado.read()
        try:
            certificado = x509.load_pem_x509_certificate(conteudo_certificado, default_backend())
        except ValueError:
            certificado = x509.load_der_x509_certificate(conteudo_certificado, default_backend())

        # Extraindo informações
        proprietario = certificado.subject
        emissor = certificado.issuer
        autoassinado = emissor == proprietario
        periodo_validade = (certificado.not_valid_before, certificado.not_valid_after)

        # Exibindo informações
        print(f"Certificado: {caminho_certificado}")
        print(f"Proprietário: {proprietario}")
        print(f"Emissor: {emissor}")
        print(f"Autoassinado: {autoassinado}")
        print(f"Período de validade: {periodo_validade[0]} até {periodo_validade[1]}")
        print("-" * 50)

    except Exception as e:
        print(f"Erro ao carregar o certificado {caminho_certificado}: {e}")

# Caminhos para os arquivos de certificado
caminhos_certificados = ['certificado1.crt', 'certificado2.crt', 'certificado3.cer']

# Executar a função para cada certificado
for caminho in caminhos_certificados:
    exibir_informacoes_certificado(caminho)


Certificado: certificado1.crt
Proprietário: <Name(C=BR,ST=Santa Catarina,L=Blumenau,O=Fundacao Universidade Regional de Blumenau FURB,CN=*.furb.br)>
Emissor: <Name(C=BR,O=Rede Nacional de Ensino e Pesquisa - RNP,CN=RNP ICPEdu OV SSL CA 2019)>
Autoassinado: False
Período de validade: 2023-07-25 17:06:03 até 2024-08-25 17:06:02
--------------------------------------------------
Certificado: certificado2.crt
Proprietário: <Name(1.3.6.1.4.1.311.60.2.1.3=BR,2.5.4.15=Private Organization,2.5.4.5=83.646.984/0006-14,C=BR,ST=Santa Catarina,L=Criciuma,O=A. Angeloni & Cia. Ltda,CN=www.angeloni.com.br)>
Emissor: <Name(C=US,ST=Arizona,L=Scottsdale,O=GoDaddy.com\, Inc.,OU=http://certs.godaddy.com/repository/,CN=Go Daddy Secure Certificate Authority - G2)>
Autoassinado: False
Período de validade: 2017-08-14 17:55:02 até 2019-08-14 17:55:02
--------------------------------------------------
Certificado: certificado3.cer
Proprietário: <Name(C=BR,ST=SC,L=Blumenau,O=Univerdade Regional de Blumenau,OU=FUR

In [9]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding as asym_padding
from cryptography.hazmat.primitives import hashes

# Função para cifrar a chave AES com a chave pública
def cifrar_chave_aes_com_chave_publica(caminho_certificado, chave_aes):
    with open(caminho_certificado, 'rb') as arquivo_certificado:
        conteudo_certificado = arquivo_certificado.read()
        certificado = x509.load_pem_x509_certificate(conteudo_certificado, default_backend())
        chave_publica = certificado.public_key()

    # Cifrar a chave AES com a chave pública
    chave_aes_cifrada = chave_publica.encrypt(
        chave_aes,
        asym_padding.OAEP(
            mgf=asym_padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    # Salvar a chave AES cifrada em um arquivo
    with open('key.aes', 'wb') as arquivo_key_aes:
        arquivo_key_aes.write(chave_aes_cifrada)

    return chave_aes_cifrada

# Caminho para o certificado que contém a chave pública
caminho_certificado = 'certificado1.crt'

# Cifrando a chave AES com a chave pública do certificado
chave_aes_cifrada = cifrar_chave_aes_com_chave_publica(caminho_certificado, chave_aes)
